In [1]:
import tensorflow as tf
print("tf version: ", tf.__version__)

import tensorflow.keras.backend  as K
import tensorflow.keras as keras

import pandas as pd
import numpy as np

#type(word_index)  dict字典
import pickle 
def save_obj(obj, file):
    with open(file, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(file ):
    with open(file, 'rb') as f:
        return pickle.load(f)

tf version:  2.3.0


In [2]:
train_x = pd.read_csv('./data/train_location.csv', usecols=['content']).values
valid_x = pd.read_csv('./data/valid_location.csv', usecols=['content']).values

train_x = [line[0].strip().split() for line in train_x]
valid_x = [line[0].strip().split() for line in valid_x]

In [3]:
# train_y = pd.read_csv('./data/train_location.csv', usecols=['label']).values+2
# valid_y = pd.read_csv('./data/valid_location.csv', usecols=['label']).values+2

#train_y = pd.read_csv('./data/train_service.csv', usecols=['label']).values+2
#valid_y = pd.read_csv('./data/valid_service.csv', usecols=['label']).values+2

# train_y = pd.read_csv('./data/train_price.csv', usecols=['label']).values+2
# valid_y = pd.read_csv('./data/valid_price.csv', usecols=['label']).values+2

#train_y = pd.read_csv('./data/train_environment.csv', usecols=['label']).values+2
#valid_y = pd.read_csv('./data/valid_environment.csv', usecols=['label']).values+2

train_y = pd.read_csv('./data/train_dish.csv', usecols=['label']).values+2
valid_y = pd.read_csv('./data/valid_dish.csv', usecols=['label']).values+2

In [4]:
cl_x = pd.read_csv('./testSetSales/chunla_filted.csv', usecols=['comment']).values
dxy_x = pd.read_csv('./testSetSales/dingxiangyuan_filted.csv', usecols=['comment']).values
jld_x = pd.read_csv('./testSetSales/jialide_filted.csv', usecols=['comment']).values
jsz_x = pd.read_csv('./testSetSales/jianshazui_filted.csv', usecols=['comment']).values
jf_x = pd.read_csv('./testSetSales/jiefu_filted.csv', usecols=['comment']).values
kl_x = pd.read_csv('./testSetSales/kuaileai_filted.csv', usecols=['comment']).values
nzn_x = pd.read_csv('./testSetSales/niuzhongniu_filted.csv', usecols=['comment']).values
sr_x = pd.read_csv('./testSetSales/shouergong_filted.csv', usecols=['comment']).values
xlk_x = pd.read_csv('./testSetSales/xiaolongkan_filted.csv', usecols=['comment']).values
zh_x = pd.read_csv('./testSetSales/zhenghuangqi_filted.csv', usecols=['comment']).values

cl_x = [line[0].strip().split() for line in cl_x]
dxy_x = [line[0].strip().split() for line in dxy_x]
jld_x = [line[0].strip().split() for line in jld_x]
jsz_x = [line[0].strip().split() for line in jsz_x]
jf_x = [line[0].strip().split() for line in jf_x]
kl_x = [line[0].strip().split() for line in kl_x]
nzn_x = [line[0].strip().split() for line in nzn_x]
sr_x = [line[0].strip().split() for line in sr_x]
xlk_x = [line[0].strip().split() for line in xlk_x]
zh_x = [line[0].strip().split() for line in zh_x]

In [5]:
comment_text = np.hstack([train_x, valid_x, cl_x, dxy_x, jld_x, jsz_x, jf_x, kl_x, nzn_x, sr_x, xlk_x, zh_x])
print(comment_text.shape)

tok_raw = tf.keras.preprocessing.text.Tokenizer()
tok_raw.fit_on_texts(comment_text)
word_index = tok_raw.word_index #语料总所有出现过词的编码

(137489,)


In [6]:
#token padding
maxlen = 200 #训练集平均词长 100

train_x = tok_raw.texts_to_sequences(train_x)
valid_x = tok_raw.texts_to_sequences(valid_x)

train_x = tf.keras.preprocessing.sequence.pad_sequences(sequences=train_x,maxlen=maxlen ,padding='post',truncating='post',value=0.0)
valid_x = tf.keras.preprocessing.sequence.pad_sequences(sequences=valid_x,maxlen=maxlen ,padding='post',truncating='post',value=0.0)

In [7]:
class blstm(tf.keras.Model): 
    def __init__(self, maxlen, embedding_matrix, embedding_dim, hidden_size, activation, output_size):
        super(blstm, self).__init__(name= 'blstm')
        self.output_size = output_size
        self.maxlen = maxlen
        self.hidden_size = hidden_size
        self.embedding_maxtrix = tf.compat.v1.get_variable(name='embedding', initializer=embedding_matrix,
                                                          dtype=tf.float64, trainable=True)
        self.blstm =  tf.keras.layers.Bidirectional(
                                tf.keras.layers.LSTM(
                                                    units=self.hidden_size,
                                                    return_sequences=True, 
                                                    return_state=True,
                                                    activation='tanh',
                                                    kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.02,l2=0.0004),
                                                    recurrent_regularizer= tf.keras.regularizers.l1_l2(l1=0.02,l2=0.0004),
                                                    bias_regularizer=tf.keras.regularizers.l1_l2(l1=0.02,l2=0.0004),
                                                    activity_regularizer= tf.keras.regularizers.l1_l2(l1=0.02,l2=0.0004),
                                                    dropout= 0.1,
                                                    recurrent_dropout= 0.1,
                                                    stateful=True)
                                                    )
        # self.flat = tf.keras.layers.Flatten()
        self.drop = tf.keras.layers.Dropout(rate=0.3, seed = 2020)
        self.dense = tf.keras.layers.Dense(
                                                units = self.output_size,
                                                activation= 'softmax',
                                                use_bias=True,
                                                kernel_initializer='glorot_uniform'
                                            )
    def call(self,inputs): #定义前向传播过程
        x_contex = tf.nn.embedding_lookup(params=self.embedding_maxtrix, ids= inputs) # (batch,maxlen, embding_size)
        x_contex = self.blstm(x_contex)[0] # 隐层 (batch,maxlen, h_size*2) 单个LSTM细胞
        # x_contex = self.flat(x_contex)
        x_contex = K.reshape(x_contex, (-1, K.shape(x_contex)[1]*K.shape(x_contex)[-1]))
        x_contex = self.drop(x_contex)
        out = self.dense(x_contex)
        return out
    
    def predict(self, x,
              batch_size=128,
              verbose=0,
              steps=None,
              callbacks=None,
              max_queue_size=10,
              workers=1,
              use_multiprocessing=False):# 父类该方法batch_size默认参数是32，所以后面调用该方法的时候，需要传入一个个性化参数
        logits = self(x) 
        fixed = tf.argmax(logits, axis=-1, output_type=tf.int32)
        return logits, fixed
        

In [8]:
#参数：
# dataset: 元组(train_x, train_y)的形式
class eval_of_experiment():
    def __init__(self, dataset, batch_size, model):
        self.dataset = dataset
        self.batch_size = batch_size
        self.model = model
        self.sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy() 
        self.precision_0 = tf.metrics.Precision(thresholds=0.5,top_k=None,class_id=0) # micro-precision
        self.precision_1 = tf.metrics.Precision(thresholds=0.5,top_k=None,class_id=1)
        self.precision_2 = tf.metrics.Precision(thresholds=0.5,top_k=None,class_id=2)
        self.precision_3 = tf.metrics.Precision(thresholds=0.5,top_k=None,class_id=3)
        self.recall_0 = tf.keras.metrics.Recall(thresholds=0.5,top_k=None,class_id=0) # micro-recall
        self.recall_1 = tf.keras.metrics.Recall(thresholds=0.5,top_k=None,class_id=1)
        self.recall_2 = tf.keras.metrics.Recall(thresholds=0.5,top_k=None,class_id=2)
        self.recall_3 = tf.keras.metrics.Recall(thresholds=0.5,top_k=None,class_id=3)
        self.y_true_label_0_num = 0
        self.y_true_label_1_num = 0
        self.y_true_label_2_num = 0
        self.y_true_label_3_num = 0

    def eval_of_valid(self):
        data = tf.data.Dataset.from_tensor_slices(self.dataset).shuffle(buffer_size=10000).batch(self.batch_size,drop_remainder=True)
        for x,y in data:
            y_prob , y_fixed = self.model.predict(x, batch_size = self.batch_size) #only predict, no training process
            self.sparse_categorical_accuracy.update_state(y_true = y, y_pred= y_prob)
            self.precision_0.update_state(y_true =tf.keras.utils.to_categorical(y), y_pred= y_prob) #必须是one-hot编码的形式
            self.precision_1.update_state(y_true =tf.keras.utils.to_categorical(y), y_pred= y_prob)
            self.precision_2.update_state(y_true =tf.keras.utils.to_categorical(y), y_pred= y_prob)
            self.precision_3.update_state(y_true =tf.keras.utils.to_categorical(y), y_pred= y_prob)
            self.recall_0.update_state(y_true =tf.keras.utils.to_categorical(y), y_pred= y_prob) 
            self.recall_1.update_state(y_true =tf.keras.utils.to_categorical(y), y_pred= y_prob) 
            self.recall_2.update_state(y_true =tf.keras.utils.to_categorical(y), y_pred= y_prob) 
            self.recall_3.update_state(y_true =tf.keras.utils.to_categorical(y), y_pred= y_prob)
            
            self.y_true_label_0_num += tf.reduce_sum(tf.cast(tf.math.equal(y,0),dtype= tf.int32)).numpy()
            self.y_true_label_1_num += tf.reduce_sum(tf.cast(tf.math.equal(y,1),dtype= tf.int32)).numpy()
            self.y_true_label_2_num += tf.reduce_sum(tf.cast(tf.math.equal(y,2),dtype= tf.int32)).numpy()
            self.y_true_label_3_num += tf.reduce_sum(tf.cast(tf.math.equal(y,3),dtype= tf.int32)).numpy()
    
    def predict_of_test(self):
        data = tf.data.Dataset.from_tensor_slices(self.dataset).batch(self.batch_size,drop_remainder=True)#不打乱
        prob=[]
        label_predic = []
        for x in data:
            y_prob , y_fixed = self.model.predict(x,batch_size=self.batch_size) # y_prob(batch,4), y_fixed(batch,)
            prob += y_prob.numpy()#tensor 转np.ndarrays
            label_predic += y_fixed.numpy()
        return prob, label_predic
            
    
    def result(self):
        total = self.y_true_label_0_num + self.y_true_label_1_num + self.y_true_label_2_num+ self.y_true_label_3_num
        p0 = self.y_true_label_0_num/total
        p1 = self.y_true_label_1_num/total
        p2 = self.y_true_label_2_num/total
        p3 = self.y_true_label_3_num/total
        print('y_true_label_0_num: %d ; rate: %f'%(self.y_true_label_0_num, p0))
        print('y_true_label_1_num: %d ; rate: %f'%(self.y_true_label_1_num,p1))
        print('y_true_label_2_num: %d ; rate: %f'%(self.y_true_label_2_num,p2))
        print('y_true_label_3_num: %d ; rate: %f'%(self.y_true_label_3_num,p3))
        print('y_true_label_num_total: %d'% (total))
        
        avg_precision = p0*self.precision_0.result()+ p1*self.precision_1.result()+ p2*self.precision_2.result()+p3*self.precision_3.result()
        avg_recall = p0*self.recall_0.result()+ p1*self.recall_1.result()+ p2*self.recall_2.result()+ p3*self.recall_3.result()
        f_0 = 2*self.precision_0.result()*self.recall_0.result()/(self.precision_0.result()+self.recall_0.result()+tf.keras.backend.epsilon())
        f_1 = 2*self.precision_1.result()*self.recall_1.result()/(self.precision_1.result()+self.recall_1.result()+tf.keras.backend.epsilon())
        f_2 = 2*self.precision_2.result()*self.recall_2.result()/(self.precision_2.result()+self.recall_2.result()+tf.keras.backend.epsilon())
        f_3 = 2*self.precision_3.result()*self.recall_3.result()/(self.precision_3.result()+self.recall_3.result()+tf.keras.backend.epsilon())
        avg_f1 = p0*f_0 + p1*f_1 + p2*f_2 + p3*f_3
        print("valid accuracy: %f" % (self.sparse_categorical_accuracy.result()))
        print("valid avg_precision: %f" % (avg_precision))
        print("valid avg_recall: %f" % (avg_recall))
        print("valid avg_f1: %f" % (avg_f1))


In [9]:
epochs = 4
batch_size = 128
learning_rate = 0.001
embedding_matrix = load_obj('./data2/train_sales_embedding_matrix.pkl')

model = blstm(
                maxlen=maxlen, 
                embedding_matrix=embedding_matrix, 
                embedding_dim=200,
                hidden_size=100,
                activation=None,
                output_size=4
            )

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [10]:
#训练过程
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy() 

for num in range(epochs):
    print('epoch %d' % (num+1))
    train = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(buffer_size=10000).batch(batch_size,drop_remainder=True)
    index = 0
    for x,y in train:
        index +=1
        with tf.GradientTape() as tape:
            labels_pred = model(x)
            #print(y.shape)
            #print(labels_pred.shape)
            loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=labels_pred)
            loss = tf.reduce_mean(loss)
            sparse_categorical_accuracy.update_state(y_true = y, y_pred= labels_pred)
            #loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=labels_pred)
            print("epoch %d; batch %d; loss %f" % (num+1, index*batch_size, loss.numpy()))
            print("epoch:%d; batch %d; train accuracy: %f" % (num+1, index*batch_size, sparse_categorical_accuracy.result()))
            grads = tape.gradient(loss, model.trainable_variables) #梯度
            optimizer.apply_gradients(grads_and_vars=zip(grads, model.trainable_variables)) #梯度和变量
    
    # 模型在训练集上 的评估参数,batch_size 128
    print('模型在训练集上的评价指标：')
    eval_train = eval_of_experiment((train_x,train_y), batch_size ,model)
    eval_train.eval_of_valid()
    eval_train.result()
    #  模型在验证集上 的评估参数，batch_size 128
    print('模型在验证集上的评价指标：')
    eval_valid = eval_of_experiment((valid_x,valid_y), batch_size ,model)
    eval_valid.eval_of_valid()
    eval_valid.result()
    
        

epoch 1

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

epoch 1; batch 128; loss 1.414243
epoch:1; batch 128; train accuracy: 0.070312
epoch 1; batch 256; loss 1.058652
epoch:1; batch 256; train accuracy: 0.324219
epoch 1; batch 384; loss 1.052985
epoch:1; batch 384; train accuracy: 0.382812
epoch 1; batch 512; loss 1.165016
epoch:1; batch 512; train accuracy: 0.419922
epoch 1; batch 640; loss 0.875773
epoch:1; batch 640; train accuracy: 0.454688
epoch 1; batch 768; loss 0.925641
epoch:1; batch 768; train accuracy: 0.472656
epoch 1; batch 896; loss 0

In [9]:
epochs = 4
batch_size = 128
learning_rate = 0.001
embedding_matrix = load_obj('./data2/train_sales_embedding_matrix.pkl')

model = blstm(
                maxlen=maxlen, 
                embedding_matrix=embedding_matrix, 
                embedding_dim=200,
                hidden_size=100,
                activation=None,
                output_size=4
            )

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [10]:
#训练过程
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy() 

for num in range(epochs):
    print('epoch %d' % (num+1))
    train = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(buffer_size=10000).batch(batch_size,drop_remainder=True)
    index = 0
    for x,y in train:
        index +=1
        with tf.GradientTape() as tape:
            labels_pred = model(x)
            #print(y.shape)
            #print(labels_pred.shape)
            loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=labels_pred)
            loss = tf.reduce_mean(loss)
            sparse_categorical_accuracy.update_state(y_true = y, y_pred= labels_pred)
            #loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=labels_pred)
            print("epoch %d; batch %d; loss %f" % (num+1, index*batch_size, loss.numpy()))
            print("epoch:%d; batch %d; train accuracy: %f" % (num+1, index*batch_size, sparse_categorical_accuracy.result()))
            grads = tape.gradient(loss, model.trainable_variables) #梯度
            optimizer.apply_gradients(grads_and_vars=zip(grads, model.trainable_variables)) #梯度和变量
    
    # 模型在训练集上 的评估参数,batch_size 128
    print('模型在训练集上的评价指标：')
    eval_train = eval_of_experiment((train_x,train_y), batch_size ,model)
    eval_train.eval_of_valid()
    eval_train.result()
    #  模型在验证集上 的评估参数，batch_size 128
    print('模型在验证集上的评价指标：')
    eval_valid = eval_of_experiment((valid_x,valid_y), batch_size ,model)
    eval_valid.eval_of_valid()
    eval_valid.result()
    
        

epoch 1

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

epoch 1; batch 128; loss 1.382526
epoch:1; batch 128; train accuracy: 0.359375
epoch 1; batch 256; loss 1.311050
epoch:1; batch 256; train accuracy: 0.378906
epoch 1; batch 384; loss 1.335639
epoch:1; batch 384; train accuracy: 0.372396
epoch 1; batch 512; loss 1.358900
epoch:1; batch 512; train accuracy: 0.363281
epoch 1; batch 640; loss 1.328960
epoch:1; batch 640; train accuracy: 0.340625
epoch 1; batch 768; loss 1.347212
epoch:1; batch 768; train accuracy: 0.322917
epoch 1; batch 896; loss 1

In [9]:
epochs = 4
batch_size = 128
learning_rate = 0.001
embedding_matrix = load_obj('./data2/train_sales_embedding_matrix.pkl')

model = blstm(
                maxlen=maxlen, 
                embedding_matrix=embedding_matrix, 
                embedding_dim=200,
                hidden_size=100,
                activation=None,
                output_size=4
            )

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [10]:
#训练过程
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy() 

for num in range(epochs):
    print('epoch %d' % (num+1))
    train = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(buffer_size=10000).batch(batch_size,drop_remainder=True)
    index = 0
    for x,y in train:
        index +=1
        with tf.GradientTape() as tape:
            labels_pred = model(x)
            #print(y.shape)
            #print(labels_pred.shape)
            loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=labels_pred)
            loss = tf.reduce_mean(loss)
            sparse_categorical_accuracy.update_state(y_true = y, y_pred= labels_pred)
            #loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=labels_pred)
            print("epoch %d; batch %d; loss %f" % (num+1, index*batch_size, loss.numpy()))
            print("epoch:%d; batch %d; train accuracy: %f" % (num+1, index*batch_size, sparse_categorical_accuracy.result()))
            grads = tape.gradient(loss, model.trainable_variables) #梯度
            optimizer.apply_gradients(grads_and_vars=zip(grads, model.trainable_variables)) #梯度和变量
    
    # 模型在训练集上 的评估参数,batch_size 128
    print('模型在训练集上的评价指标：')
    eval_train = eval_of_experiment((train_x,train_y), batch_size ,model)
    eval_train.eval_of_valid()
    eval_train.result()
    #  模型在验证集上 的评估参数，batch_size 128
    print('模型在验证集上的评价指标：')
    eval_valid = eval_of_experiment((valid_x,valid_y), batch_size ,model)
    eval_valid.eval_of_valid()
    eval_valid.result()
    
        

epoch 1

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

epoch 1; batch 128; loss 1.407082
epoch:1; batch 128; train accuracy: 0.117188
epoch 1; batch 256; loss 1.143226
epoch:1; batch 256; train accuracy: 0.367188
epoch 1; batch 384; loss 0.948409
epoch:1; batch 384; train accuracy: 0.476562
epoch 1; batch 512; loss 1.032946
epoch:1; batch 512; train accuracy: 0.521484
epoch 1; batch 640; loss 0.973398
epoch:1; batch 640; train accuracy: 0.550000
epoch 1; batch 768; loss 0.984126
epoch:1; batch 768; train accuracy: 0.561198
epoch 1; batch 896; loss 1